# CardioML

Elaborato di Programmazione di Applicazioni Data Intensive - 2023

Membri del Gruppo:
- Matteo Iorio
- Fabio Vincenzi
- Stefano Furi

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="ticks")
sns.set_style("whitegrid")

## Creazione del DataFrame

Ci siamo posti l'obbiettivo di realizzare un progetto che **possa veramente risultare utile** per alcuni professionisti del settore. Quello che andremo a studiare è un problema che riguarda i fanciulli di tenera età, più in particolare andremo ad analizzare i *risultati di rilevazioni cardiache*, andando a determinare se avranno o meno complicanze cardiache patologiche. 

Il dataset utilizzato riguarda una serie di misurazioni effettuate attraverso [cardiotocografia](https://it.wikipedia.org/wiki/Cardiotocografia), consultabile presso [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/cardiotocography#).



Lo scopo di questo elaborato verrà scritto qui molto presto :)

Decidiamo se mettere il `setup.py` con link qui direttamente alla repo github, così se i prof chiedono solo il file jupyter, possono tranquillamente usarlo senza clonare l'intera repo, mentre invece per i temerari che clonano l'intera repo, il dataset è già qui integrato.

## Preparazione del dataset

In [2]:
raw_df = pd.read_csv("../data/ctg.csv", delimiter=';')

In [3]:
raw_df.head()

,FileName,Date,SegFile,b,e,LBE,LB,AC,FM,UC,...,C,D,E,AD,DE,LD,FS,SUSP,CLASS,NSP
0,Variab10.txt,01/12/96,CTG0001.txt,240,357,120,120,0,0,0,...,0,0,0,0,0,0,1,0,9,2
1,Fmcs_1.txt,03/05/96,CTG0002.txt,5,632,132,132,4,0,4,...,0,0,0,1,0,0,0,0,6,1
2,Fmcs_1.txt,03/05/96,CTG0003.txt,177,779,133,133,2,0,5,...,0,0,0,1,0,0,0,0,6,1
3,Fmcs_1.txt,03/05/96,CTG0004.txt,411,1192,134,134,2,0,6,...,0,0,0,1,0,0,0,0,6,1
4,Fmcs_1.txt,03/05/96,CTG0005.txt,533,1147,132,132,4,0,5,...,0,0,0,0,0,0,0,0,2,1


SPIEGAZIONE RAPIDA DELLE VARIABILI, DIVISE PER CATEGORICHE E CONTINUE

Alcune variabili categoriche sono state mappate a `np.float64`, mentre altre sarebbe stato sufficiente un tipo `np.int32`. Possiamo quindi ricaricare il dataset, specificando il tipo di dato per ogni colonna, facendo anche riferimento all'utilizzo di memoria

In [4]:
raw_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2126 entries, 0 to 2125
Data columns (total 40 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   FileName  2126 non-null   object
 1   Date      2126 non-null   object
 2   SegFile   2126 non-null   object
 3   b         2126 non-null   int64 
 4   e         2126 non-null   int64 
 5   LBE       2126 non-null   int64 
 6   LB        2126 non-null   int64 
 7   AC        2126 non-null   int64 
 8   FM        2126 non-null   int64 
 9   UC        2126 non-null   int64 
 10  ASTV      2126 non-null   int64 
 11  MSTV      2126 non-null   object
 12  ALTV      2126 non-null   int64 
 13  MLTV      2126 non-null   object
 14  DL        2126 non-null   int64 
 15  DS        2126 non-null   int64 
 16  DP        2126 non-null   int64 
 17  DR        2126 non-null   int64 
 18  Width     2126 non-null   int64 
 19  Min       2126 non-null   int64 
 20  Max       2126 non-null   int64 
 21  Nmax      2126

Possiamo poi procedere con la classificazione dei nostro dati, più nello specifico abbiamo individuato 3 macro classi:
* *bool_vars*: Tutte le variabili del nostro data frame che assumono valori tra [0, 1], andremo ad assegnare come tipologia di valore quello *booleano*
* *cat_varts*: All'interno di questa tipologia di variabili, troviamo quelle *categoriche*, assegneremo a tali variabili il valore *category*
* *num_vars*: In quest'ultima tipologia di variabili, abbiamo solamente le tipologie di valori *numerici*, le quali verranno trattate come variabili di tipo *float32*


In [5]:
bool_vars = [
    "A", "B", "C", "D", "SH",
    "AD", "DE", "LD", "FS", "SUSP"
]

cat_vars = ["CLASS", "NSP", "Tendency"]

num_vars = [
    "b", "e", "LBE", "LB", "AC",
    "FM", "UC", "ASTV",
    "MSTV", "ALTV", "MLTV", "DL",
    "DS", "DP", "DR", "Width", "Min",
    "Max", "Nmax", "Nzeros", "Mode",
    "Mean", "Median", "Variance"
]

custom_dtpes = { 
    **{ col: "bool" for col in bool_vars }, 
    **{ col: "category" for col in cat_vars},
    **{ col: "float32" for col in num_vars}
}

Possiamo quindi procedere a leggere nuovamente il Data Frame, AGGIUNGERE 

In [6]:
raw_df = pd.read_csv("../data/ctg.csv", delimiter=';', dtype=custom_dtpes, decimal=',')

OTTIMIZZAZIONE

In [7]:
raw_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2126 entries, 0 to 2125
Data columns (total 40 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   FileName  2126 non-null   object  
 1   Date      2126 non-null   object  
 2   SegFile   2126 non-null   object  
 3   b         2126 non-null   float32 
 4   e         2126 non-null   float32 
 5   LBE       2126 non-null   float32 
 6   LB        2126 non-null   float32 
 7   AC        2126 non-null   float32 
 8   FM        2126 non-null   float32 
 9   UC        2126 non-null   float32 
 10  ASTV      2126 non-null   float32 
 11  MSTV      2126 non-null   float32 
 12  ALTV      2126 non-null   float32 
 13  MLTV      2126 non-null   float32 
 14  DL        2126 non-null   float32 
 15  DS        2126 non-null   float32 
 16  DP        2126 non-null   float32 
 17  DR        2126 non-null   float32 
 18  Width     2126 non-null   float32 
 19  Min       2126 non-null   float32 
 20  Max     

In questo modo, abbiamo ridotto di quasi la metà la quantità di memoria utilizzata per mantenere il dataset.

Mappiamo i nomi di aclune colonne in nomi più esplicativi.

In [8]:
col_names = {
    "FileName": "file_name",
    "Date": "date",
    "SegFile": "seg_file",
    "b": "start_instant",
    "e": "end_instant",
    "LBE": "medical_baseline",
    "LB": "sisproto_baseline",
    "AC": "heart_acc",
    "FM": "foetal_move",
    "UC": "utherine_contract",
    "ASTV": "pct_short_variability",
    "MSTV": "mean_short_variability",
    "ALTV": "pct_long_variability",
    "MLTV": "mean_long_variability",
    "DL": "light_dec",
    "DS": "severe_dec",
    "DP": "prolongued_dec",
    "DR": "repetitive_dec",
    "A": "calm_sleep",
    "B": "rem_sleep",
    "C": "calm_vigilance",
    "D": "active_vigilance",
    "SH": "shift_pattern",
    "AD": "acc_pattern_stress", # negativo se pattern in decelerazione
    "DE": "dec_pattern_vagal",
    "LD": "largely_dec",
    "FS": "patho_flat_sinusoidal",
    "SUSP": "suspect"
}

In [9]:
raw_df.rename(columns=col_names, inplace=True)
raw_df.columns = raw_df.columns.str.lower()
raw_df.columns

Index(['file_name', 'date', 'seg_file', 'start_instant', 'end_instant',
       'medical_baseline', 'sisproto_baseline', 'heart_acc', 'foetal_move',
       'utherine_contract', 'pct_short_variability', 'mean_short_variability',
       'pct_long_variability', 'mean_long_variability', 'light_dec',
       'severe_dec', 'prolongued_dec', 'repetitive_dec', 'width', 'min', 'max',
       'nmax', 'nzeros', 'mode', 'mean', 'median', 'variance', 'tendency',
       'calm_sleep', 'rem_sleep', 'calm_vigilance', 'active_vigilance', 'e',
       'acc_pattern_stress', 'dec_pattern_vagal', 'largely_dec',
       'patho_flat_sinusoidal', 'suspect', 'class', 'nsp'],
      dtype='object')

In [10]:
raw_df.head(5)

,file_name,date,seg_file,start_instant,end_instant,medical_baseline,sisproto_baseline,heart_acc,foetal_move,utherine_contract,...,calm_vigilance,active_vigilance,e,acc_pattern_stress,dec_pattern_vagal,largely_dec,patho_flat_sinusoidal,suspect,class,nsp
0,Variab10.txt,01/12/96,CTG0001.txt,240.0,357.0,120.0,120.0,0.0,0.0,0.0,...,False,False,0,False,False,False,True,False,9,2
1,Fmcs_1.txt,03/05/96,CTG0002.txt,5.0,632.0,132.0,132.0,4.0,0.0,4.0,...,False,False,0,True,False,False,False,False,6,1
2,Fmcs_1.txt,03/05/96,CTG0003.txt,177.0,779.0,133.0,133.0,2.0,0.0,5.0,...,False,False,0,True,False,False,False,False,6,1
3,Fmcs_1.txt,03/05/96,CTG0004.txt,411.0,1192.0,134.0,134.0,2.0,0.0,6.0,...,False,False,0,True,False,False,False,False,6,1
4,Fmcs_1.txt,03/05/96,CTG0005.txt,533.0,1147.0,132.0,132.0,4.0,0.0,5.0,...,False,False,0,False,False,False,False,False,2,1


Ai fini dello scopo di questo progetto, le colonne `file_name`, `seg_file` e `date` non risultano particolarmente utili per la creazione di modelli predittivi.

In [11]:
df = raw_df.drop(columns=['file_name', 'date', 'seg_file'])

Il dataset è ora utilizzabile per procedere con l'analisi esplorativa.

## Analisi Esplorativa

Possiamo rapidamente vedere quali variabili del dataset hanno correlazione maggiore, per poter poi comprendere come effettivamente alcuni valori possano influenzare altri. Utilizziamo quindi l'indice di correlazione di Pearson.

Text(0.5, 1.0, 'Correlation Matrix Heatmap')

In [13]:
corr_matrix = df.drop(columns=['min', 'mean', 'max','nmax', 'mode', 'median']) \
    .corr(method="pearson")

plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, cmap="coolwarm")
plt.title("Correlation Matrix Heatmap")

Creiamo una **color map** che ci permetterà di rendere i nostri grafici più chiari. La nostra *color map* si basa su una specifica colonna del nostro *data frame*, la colonna **NSP**. All'interno del data frame, la colonna assueme semplicemente 3 valori.

Come si può notare dalla heatmap, esiste una una forte correlazione tra la variabile `width` e `mean_short_variability`, pari a:

In [68]:
corr_matrix['width']['mean_short_variability']

0.6608467767745067

In [ ]:
sns.scatterplot(data=df, x='width', y='mean_short_variability', hue='nsp', palette=['#6c9a8b', '#ffc914', '#d62828'])

La **"width"** rappresenta la durata delle onde nell'attività cardiaca fetale registrata. Potrebbe essere calcolata come la durata dell'onda P, la durata dell'onda QRS o altre onde specifiche nell'ECG fetale. La larghezza dell'onda ci permette di ottenere delle informazioni riguardo la conducibilità e sulla morfologia dell'attività cardiaca del neonato.

La **"mean_short_variability"** rappresenta invece la variabilità media a breve termine dell'attività cardiaca fetale. Questa variabile fa riferimento alla fluttuazione delle frequenze cardiache nel breve termine, calcolata come i battiti cardiaci consecutivi nell'arco di pochi minuti. La grande variabilità a breve termine è considerata un fattore di salute del neonato, dal momento in cui la variabilità è associata ad una migliore adattabilità del feto.

In generale, alcune delle anomalie cardiache nei neonati riescono influenzare sia la *"width"* che la *"mean_short_variability"* registrate durante questa analisi, la cardiotocografia. Ad esempio:

* **Stenosi valvolare**: Questa malattia può influire sulla tipologia delle onde registrate nell'attività cardiaca fetale, determinando una crescita della *"width"*.

* **Anomalia del ritmo cardiaco**: Queste anomalie possono anche influenzare la variabilità a breve termine, riducendola o causando un pattern irregolare nella *width*.

* **Malformazioni cardiache strutturali**: Le malformazioni cardiache strutturali possono influire sulla forma delle onde nell'attività cardiaca del nenonato sotto osservazione, determinando una *"width"* alterata.

Come si può notare dalla heatmap, esiste una una forte correlazione tra la variabile `mean_short_variability` e `variance`, pari a:

In [66]:
corr_matrix['mean_short_variability']['variance']

0.5558524410341099

In [ ]:
sns.scatterplot(data=df, x='mean_short_variability', y='variance', hue='nsp', palette=['#6c9a8b', '#ffc914', '#d62828'])

La **"variance"** è una misura statistica che rappresenta la dispersione dei dati intorno alla media.

Una *correlazione positiva* tra queste due variabili suggerisce che una maggiore *"variance"* è associata a una maggiore *"mean_short_variability"*. Ciò indica che le oscillazioni nella frequenza cardiaca fetale sono più variabili e presentano una maggiore dispersione intorno alla media durante il breve periodo di analisi.

Come si può notare dalla heatmap, esiste una una forte correlazione tra la variabile `largely_dec` e `prolongued_dec`, pari a:

In [ ]:
corr_matrix['largely_dec']['prolongued_dec']

Possiamo quindi effettuare un grafico per visualizzare la distribuzione dei valori (diagonale) e come si presenta l'output di `NSP` in base alla distribuzione dei due valori

In [ ]:
sns.pairplot(data=df, vars=['largely_dec', 'prolongued_dec'], hue='nsp',
             palette=['#6c9a8b', '#ffc914', '#d62828'],
             diag_kind="kde", plot_kws={'alpha': 0.6, 's': 30})